In [ ]:
import pandas as pd


# Lista de provincias válidas
PROVINCIAS_VALIDAS = [
    'AZUA', 'BAHORUCO', 'BARAHONA', 'DAJABON', 'DUARTE', 
    'ELIAS PIÑA', 'EL SEIBO', 'ESPAILLAT', 'HATO MAYOR', 
    'HERMANAS MIRABAL', 'INDEPENDENCIA', 'LA ALTAGRACIA', 
    'LA ROMANA', 'LA VEGA', 'MARIA TRINIDAD SANCHEZ', 
    'MONSEÑOR NOUEL', 'MONTE CRISTI', 'MONTE PLATA', 
    'PEDERNALES', 'PERAVIA', 'PUERTO PLATA', 'SAMANA', 
    'SAN CRISTOBAL', 'SAN JOSE DE OCOA', 'SAN JUAN', 
    'SAN PEDRO DE MACORIS', 'SANCHEZ RAMIREZ', 'SANTIAGO', 
    'SANTIAGO RODRIGUEZ', 'SANTO DOMINGO', 'VALVERDE', 
    'DISTRITO NACIONAL'
]


# FUNCIÓN PARA PROCESAR UN AÑO

def procesar_año(archivo, año):
    """

    Parámetros:
    - archivo: ruta del archivo CSV
    - año: año de los datos 
    
    """
    # Leer archivo
    df = pd.read_csv(archivo, skiprows=3, encoding='ISO-8859-1')
    
    # Renombrar primera columna
    df.rename(columns={'Unnamed: 0': 'Provincia'}, inplace=True)
    
    # Corrección para 2025 (problema de encoding)
    if año == 2025:
        df['Provincia'] = df['Provincia'].str.replace('Ã', 'Ñ')
    
    # Filtrar provincias válidas
    df_filtrado = df[df['Provincia'].isin(PROVINCIAS_VALIDAS)]
    
    # Extraer partos vaginales (columnas 0-2)
    df_vaginales = df_filtrado.iloc[:, 0:3].copy()
    df_vaginales['Tipo de Parto'] = 'Vaginal'
    df_vaginales.rename(columns={'<15 A.': 'Menor de 15'}, inplace=True)
    df_vaginales.rename(columns={df_vaginales.columns[2]: '15-19'}, inplace=True)
    
    # Extraer partos por cesáreas
    # Nota: 2023 tiene índices diferentes (11, 12) vs otros años (10, 11)
    if año == 2023:
        columnas_cesareas = [0, 11, 12]
    else:
        columnas_cesareas = [0, 10, 11]
    
    df_cesareas = df_filtrado.iloc[:, columnas_cesareas].copy()
    df_cesareas['Tipo de Parto'] = 'Cesarea'
    df_cesareas.rename(columns={df_cesareas.columns[1]: 'Menor de 15'}, inplace=True)
    df_cesareas.rename(columns={df_cesareas.columns[2]: '15-19'}, inplace=True)
    
    # Combinar vaginales y cesáreas
    df_combinado = pd.concat([df_vaginales, df_cesareas], ignore_index=True)
    
    # Transformar a formato largo
    df_largo = df_combinado.melt(
        id_vars=['Provincia', 'Tipo de Parto'],
        value_vars=['Menor de 15', '15-19'],
        var_name='Rango de Edad',
        value_name='Cantidad'
    )
    
    # Limpiar columna Cantidad
    if año == 2023:
        df_largo['Cantidad'] = df_largo['Cantidad'].str.replace(',', '')
    
    df_largo['Cantidad'] = df_largo['Cantidad'].astype('int64')
    
    # Normalizar nombres de provincias
    df_largo['Provincia'] = df_largo['Provincia'].str.title()
    
    # Agregar columna de año
    df_largo['Año'] = año
    
    return df_largo


# FUNCIÓN PARA PROCESAR AÑOS TRIMESTRALES

def procesar_año_trimestral(archivos, año):
    
    # Leer todos los trimestres
    dfs_trimestres = []
    for archivo in archivos:
        df = pd.read_csv(archivo, skiprows=3, encoding='ISO-8859-1')
        dfs_trimestres.append(df)
    
    # Concatenar trimestres
    df_completo = pd.concat(dfs_trimestres, ignore_index=True)
    
    # Renombrar primera columna
    df_completo.rename(columns={'Unnamed: 0': 'Provincia'}, inplace=True)
    
    # Corrección para 2025 (problema de encoding)
    if año == 2025:
        df_completo['Provincia'] = df_completo['Provincia'].str.replace('Ã', 'Ñ')
    
    # Filtrar provincias válidas
    df_filtrado = df_completo[df_completo['Provincia'].isin(PROVINCIAS_VALIDAS)]
    
    # Extraer partos vaginales (columnas 0-2)
    df_vaginales = df_filtrado.iloc[:, 0:3].copy()
    df_vaginales['Tipo de Parto'] = 'Vaginal'
    df_vaginales.rename(columns={'<15 A.': 'Menor de 15'}, inplace=True)
    df_vaginales.rename(columns={df_vaginales.columns[2]: '15-19'}, inplace=True)
    
    # Extraer partos por cesáreas
    df_cesareas = df_filtrado.iloc[:, [0, 10, 11]].copy()
    df_cesareas['Tipo de Parto'] = 'Cesarea'
    df_cesareas.rename(columns={df_cesareas.columns[1]: 'Menor de 15'}, inplace=True)
    df_cesareas.rename(columns={df_cesareas.columns[2]: '15-19'}, inplace=True)
    
    # Combinar vaginales y cesáreas
    df_combinado = pd.concat([df_vaginales, df_cesareas], ignore_index=True)
    
    # Transformar a formato largo
    df_largo = df_combinado.melt(
        id_vars=['Provincia', 'Tipo de Parto'],
        value_vars=['Menor de 15', '15-19'],
        var_name='Rango de Edad',
        value_name='Cantidad'
    )
    
    # Limpiar columna Cantidad
    df_largo['Cantidad'] = df_largo['Cantidad'].astype('int64')
    
    # Normalizar nombres de provincias
    df_largo['Provincia'] = df_largo['Provincia'].str.title()
    
    # Agregar columna de año
    df_largo['Año'] = año
    
    # Agrupar (sumar cantidades duplicadas de trimestres)
    df_agrupado = df_largo.groupby(
        ['Provincia', 'Tipo de Parto', 'Rango de Edad', 'Año']
    )['Cantidad'].sum().reset_index()
    
    return df_agrupado


# PROCESAR AÑOS INDIVIDUALES (2020, 2021, 2023)


print("Procesando años individuales...")
df_2020 = procesar_año('INFORME 2020.csv', 2020)
df_2021 = procesar_año('INFORME 2021.csv', 2021)
df_2023 = procesar_año('INFORME 2023.csv', 2023)
print("Años individuales procesados")


# PROCESAR AÑOS CON TRIMESTRES (2022, 2024, 2025)


print("Procesando años trimestrales...")

# Procesar 2022
archivos_2022 = [
    'INFORME PRIMER TRIMESTRE 2022.csv',
    'INFORME SEGUNDO TRIMESTRE 2022.csv',
    'INFORME TERCER TRIMESTRE 2022.csv',
    'INFORME CUARTO TRIMESTRE 2022.csv'
]
df_2022 = procesar_año_trimestral(archivos_2022, 2022)
print("2022 procesado")

# Procesar 2024
archivos_2024 = [
    'INFORME PRIMER TRIMESTRE 2024.csv',
    'INFORME SEGUNDO TRIMESTRE 2024.csv',
    'INFORME TERCER TRIMESTRE 2024.csv',
    'INFORME CUARTO TRIMESTRE 2024.csv'
]
df_2024 = procesar_año_trimestral(archivos_2024, 2024)
print("2024 procesado")

# Procesar 2025
archivos_2025 = [
    'INFORME PRIMER TRIMESTRE 2025.csv',
    'INFORME SEGUNDO TRIMESTRE 2025.csv',
    'INFORME TERCER TRIMESTRE 2025.csv'
]
df_2025 = procesar_año_trimestral(archivos_2025, 2025)
print("2025 procesado")

print("Todos los años trimestrales procesados")

# COMBINAR TODOS LOS AÑOS

df_final = pd.concat([df_2020, df_2021, df_2022, df_2023, df_2024, df_2025])

# Exportar a csv 

df_final.to_csv('Partos Adolescentes 2020-2025.csv', index=False)
print("Archivo 'Partos Adolescentes 2020-2025.csv' exportado exitosamente")

Procesando años individuales...


C:\Users\pulum\AppData\Local\Temp\ipykernel_3876\3362038572.py:30: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, skiprows=3, encoding='ISO-8859-1')


Años individuales procesados
Procesando años trimestrales...
2022 procesado
2024 procesado
2025 procesado
Todos los años trimestrales procesados
Archivo 'Partos Adolescentes 2020-2025.csv' exportado exitosamente
